* [건강정보 > 병(의)원 정보 > 병(의)원 찾기 | 건강iN](https://hi.nhis.or.kr/ca/ggpca001/ggpca001_m03.do)

In [ ]:
import requests
import pandas as pd
import numpy as np
import json
import time

https://hi.nhis.or.kr/ca/ggpca001/ggpca001_p06.do

In [ ]:
url = "https://hi.nhis.or.kr/ca/ggpca001/ggpca001_p06.do"
params = "isMobile=&pageNum=1&pageFirst=Y&pageSize=3&viewType=&ykiho="
# params = params + "&cur_sido_nm=&cur_sigungu_nm=&cur_emd_nm=&cur_sido=&cur_sigungu=&cur_emd="
# params = params + "&search_sido_nm=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&search_sigungu_nm=%EA%B0%95%EB%82%A8%EA%B5%AC"
# params = params + "&search_emd_nm=&search_road_nm=&latitude=37.3246580766&longitude=127.9863282634&lat=&lng=&radius=1000&gpsYn=N&search_sido=11&search_sigungu=680&search_emd_gubun=emd&search_emd=&search_road=&search_name=&searchType=&searchClinicCode=&searchSpcClinicType=02&searchSpcClinicSrchCd=&searchPharmacyDayType="

In [ ]:
parameter = {}
for p in params.split("&")[1:]:
    key = p.split("=")[0]
    val = p.split("=")[1]
    parameter[key] = val

parameter

In [ ]:
# pagination 에 보여줄 페이지 갯수이기 때문에 변경해도 크롤링에는 전혀 영향이 없음
parameter["pageSize"] = 3

In [ ]:
# 찾을 시도의 이름으로 인코딩을 해서 전달해야함
parameter['search_sido_nm'] = ''

In [ ]:
def get_page(page_num):
    parameter["pageNum"] = page_num
    response = requests.post(url, parameter)
    data = json.loads(response.text)
    return data

data = get_page(2)
data["totalCount"]

In [ ]:
# 마지막 페이지 번호를 찾을 수 있습니다.
data["pagination"]

In [ ]:
df = pd.DataFrame(data["list"])
df.shape

In [ ]:
df.head()

In [ ]:
pd.DataFrame(data["list"]).values

In [ ]:
# 전체 데이터 수에서 한페이지를 가져올 때 10개의 데이터를 가져오기 때문에 10으로 나눠준다.
# 소수점은 버리고 한페이지가 더 있을 수 있기 때문에 1을 더해준다.
page_count = (data["totalCount"] // 10) + 1
# page_count = 5
page_count

In [ ]:
from tqdm import tqdm
page_list = []
for page_num in tqdm(range(2315, page_count)):
    # 서버의 부담을 줄이기 위해 약간의 시간차를 두고 가져옵니다.
    time.sleep(np.random.uniform(0, 1))
    data = get_page(page_num)
    list_value = pd.DataFrame(data["list"]).values
    page_list.extend(list_value)
    # 크롤링이 끝나고 저장하는 방법도 있지만 트래픽이 끊기거나 오류 등으로 인해 중단될 수도 있기 때문에 중간 중간 저장합니다.
    if page_num % 10 == 0:
        df = pd.DataFrame(page_list)
        df.to_csv("page_list_2.csv", index=False)

In [ ]:
page_count

In [ ]:
df.to_csv("page_list_2.csv", index=False)

In [ ]:
df_pre = pd.read_csv("page_list_2315.csv", low_memory=False)
df_pre.head()

In [ ]:
df.head()

In [ ]:
df_all = pd.concat([df_pre, df])
df_all.shape

In [ ]:
df_all

In [ ]:
columns = pd.DataFrame(data["list"]).columns
columns

In [ ]:
df.head()

In [ ]:
df["SI_DO_CD"].value_counts()

In [ ]:
df["VIEW_TYPE"].unique()

In [ ]:
df["CLSFC_CD_DESC"].value_counts()

In [ ]:
import csv
with open('hospital_list.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(page_list)

In [ ]:
pd.read_csv("hospital_list.csv").shape

In [ ]:
df.to_csv("hospital_list.csv", index=False)